In [1]:
import pandas as pd
import sys
import requests
sys.path.append(r'/home/jupyter/reusable_code')
import google_api_functions as gaf
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

from pandas import read_csv

from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf




from pandas.plotting import autocorrelation_plot
from matplotlib import pyplot
import itertools

import numpy as np
import re

from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Set up modules for Google functionality
from google.cloud import bigquery # To run BQ statements
from google_auth_oauthlib import flow # To authorise as user
from googleapiclient.discovery import build # To pull in from sheets, slides etc. API
from google.auth.transport.requests import Request
from google.cloud.bigquery import magics
import google

from datetime import datetime, timedelta
from matplotlib import pyplot as plt

In [2]:
def Interpolate_Missing (df,freq='d'):
    df=df.reindex(pd.date_range(start=df.index.min(), end=df.index.max(),freq=freq))
    df=df.interpolate()
    return df

def Cap_data (ser,show_info=True):
    
    newSer=ser.copy() # Make a deep copy of the series to manipulate, without affecting original series
    
    if type(ser) is pd.core.series.Series : # Check if series
        skewvalue=ser.skew()
        print('The skew is {:.2f}. A value of <-1 indicates lots of small values, and a value of >1 indicates the presence of some large values.'.format(skewvalue))
        
        if show_info==True:
            print(ser.describe())
            ser.hist()
            plt.show()
        if skewvalue>1:
            threshold=newSer.quantile(0.99)# For lots of large values, consider the 99th percentile as the threshold
            mask=newSer>threshold # Create the boolean mask for all the records exceeding the threshold
            
            affected_records=mask[mask==True].size # Identify how many records affected
            series_size=newSer.size # How many records in dataset
            pc_affected=float(affected_records)/float(series_size)*100
            
            newSer=newSer.mask(newSer>threshold,threshold)  # Apply the mask
            print('The 99th percentile is {}. Values greater than this will be capped, amounting to {:.0f} records and {:.1f}% of the dataset.'.format(threshold,affected_records,pc_affected))
        elif skewvalue<-1:
            threshold=newSer.quantile(0.01)[columnname] # For lots of large values, consider the 99th percentile as the threshold
            mask=newSer<threshold # Create the boolean mask for all the records exceeding the threshold
            
            affected_records=mask[mask==True].size # Identify how many records affected
            series_size=newSer.size # How many records in dataset
            pc_affected=float(affected_records)/float(series_size)*100
            #newSer= np.where(newSer<threshold, threshold,newSer)  # Apply the mask
            newSer=newSer.mask(newSer<threshold,threshold)
            print('The 1st percentile is {}. Values less than this will be capped, amounting to {:.0f} records and {:.1f}% of the dataset.'.format(threshold,affected_records,pc_affected))
            
        else:
            print('Skew is ok, no capping applied')
            
        newskew=newSer.skew()
        print('The skew is now {:.2f}.'.format(newskew))
        
        if show_info==True:
            print(newSer.describe())
            newSer.hist()  
            plt.show()
        
        
    
    elif type(ser) is pd.core.frame.DataFrame and ser.shape[1]==1: # Check if one-column dataframe, if so take the first column and reference throughout for the processing
        columnname=ser.columns[0] #Get the name of the series to use later
        skewvalue=ser[columnname].skew()
        print('The skew is {:.2f}. A value of <-1 indicates lots of small values, and a value of >1 indicates the presence of some large values.'.format(skewvalue))
        
        if show_info==True:
            print(ser.describe())
            ser.hist()
            plt.show()
        if skewvalue>1:
            threshold=newSer.quantile(0.99)[columnname] # For lots of large values, consider the 99th percentile as the threshold
            mask=newSer[columnname]>threshold # Create the boolean mask for all the records exceeding the threshold
            print(mask[columnname==True])
            affected_records=mask[mask==True].size # Identify how many records affected
            series_size=newSer.size # How many records in dataset
            pc_affected=float(affected_records)/float(series_size)*100
            newSer[columnname]= np.where(mask, threshold,newSer[columnname])  # Apply the mask
            print('The 99th percentile is {}. Values greater than this will be capped, amounting to {:.0f}/{:.0f} records ({:.1f}% of the dataset).'.format(threshold,affected_records,series_size,pc_affected))
        
        elif skewvalue<-1:
            threshold=newSer.quantile(0.01)[columnname] # For lots of large values, consider the 99th percentile as the threshold
            mask=newSer[columnname]<threshold # Create the boolean mask for all the records exceeding the threshold
            print(mask[columnname==True])
            affected_records=mask[mask==True].size # Identify how many records affected
            series_size=newSer.size # How many records in dataset
            pc_affected=float(affected_records)/float(series_size)*100
            newSer[columnname]= np.where(mask, threshold,newSer[columnname])  # Apply the mask
            print('The 1st percentile is {}. Values less than this will be capped, amounting to {:.0f}/{:.0f} records ({:.1f}% of the dataset).'.format(threshold,affected_records,series_size,pc_affected))
        
        else:
            print('Skew is ok, no capping applied')
            
        newskew=newSer[columnname].skew()
        print('The skew is now {:.2f}.'.format(newskew))
        
        if show_info==True:
            print(newSer[columnname].describe())
            newSer.hist()  
            plt.show()

    else:
        print('Can''t run data, needs to be a 1 column dataframe or a series')
    return newSer

def generate_test_trains(df,HoldoutPC=0.05,recenttrain=90,lowerCutoff=None,upperCutoff=None):
    
    if lowerCutoff:
        df=df[lowerCutoff:]
    if upperCutoff:
        df=df[:upperCutoff]
    
    # This function splits the samples into three groups. A holdout is taken from the most recent data, used to evaluate model fit. The proportion to hold out is determined by HoldoutPC    
    # Two different training sets are generated. One uses all available data, the other uses the most recent N periods, on the proviso that older day may no longer be relevant so models 
    # trained on this data may be better
    
    # If for whatever reason we wish to trim the edges (i.e. we want to build a model for "last week", this can be done using the lower- and upperCutoff)
    
    holdoutsize=int(np.floor(len(df)*HoldoutPC))
    holdout=df[-holdoutsize:]
    print('{} records used as a test dataset ranging from {} and {}.'.format(holdoutsize,holdout.index.min() ,holdout.index.max()))
    fullTrain=df[:-holdoutsize]
    print('{} records used in the "Full Training" dataset ranging from {} and {}.'.format(len(fullTrain),fullTrain.index.min() ,fullTrain.index.max()))
    recentTrain=fullTrain[-recenttrain:]
    print('{} records used in the "Recent Data" training dataset ranging from {} and {}.'.format(len(recentTrain),recentTrain.index.min() ,recentTrain.index.max()))
    return(holdout,fullTrain,recentTrain)

def timeSeries_diagnostics(ser,hypothesised_seasonality=7):
    if type(ser) is pd.core.series.Series :  
        print("Correlation at lag 1 is :",ser.autocorr())

        # Correlation with itself (7 days ago)
        print("Correlation at lag {} is :".format(hypothesised_seasonality),ser.autocorr(hypothesised_seasonality))

        xcutoff=hypothesised_seasonality*3+1 # Show three seasonal periods to see if that is validated
        plot_acf(ser, lags=xcutoff)

        
        # Get Partial ACF

        PACF_values=pd.DataFrame(pacf(ser,nlags=xcutoff)) #PACF numpy array converted to df
        #PACF_values.plot() #Plot
        #pyplot.show()
        plot_pacf(ser)
        print("Reminder- the point at which PACF drops to 0 is the point at which no further value is added by this lag over and above the points before it")

    else:
        print('Please pass a Pandas Series for evaluation')

def HW_grid(seasonal=[7]):
    Initial_grid=list(itertools.product(['add','mul'],['add','mul',None],seasonal))
    Deduped_grid=list(set([(i[0],i[1],i[2]) if i[1]!=None else (i[0],i[1],0) for i in Initial_grid ])) # If seasonality is none, we don't need the individual variantsv `    
     #Create all 6 combos of additive, multiplicative multiplied by the seasonal
    return Deduped_grid


def Remove_duplicate_training_sets(training_sets):
    if type(training_sets)!=dict:
        print ('ERROR: training_sets parameter needs to take the form of a dictionary {label: pandas series}')
    else:
        del_dict={x:False for x in training_sets} #Create a dictionary saying which sets to delete, default to None
        for n1,x in enumerate(training_sets): # Loop training sets
            for n2,y in enumerate(training_sets): # For each set
                if x==y or n2<n1: # Don't check references to self and only look at outstanding combinations
                    pass
                else:
                    print('Comparing {} with {}'.format(x,y))
                    if training_sets[x].equals(training_sets[y]): # If any other series are duplicate
                        del_dict[y]=True # Mark them for deletion
        # Now actually delete if appropriate
        for i in del_dict:
            if del_dict[i]==True:
                del training_sets[i]
    return training_sets

def error_calcs(actual, estimated): 
    actual, estimated = np.array(actual), np.array(estimated)
    nobs=len(actual)
    
    fit_dict={'MAPE':np.mean(np.abs(actual - estimated) / actual) ,
             'MSE':np.mean((actual - estimated)**2),
             'RMSE':np.sqrt(np.mean((actual - estimated)**2)),
             'MAE':np.mean(np.abs(actual - estimated)),
             'Correlation': np.corrcoef(actual , estimated)[0][1]}
    
    return fit_dict

### 1) Get Time Series data. This should be a Pandas series, with the datetime as the index.

In [22]:
# Free Trial Starts Time Series
creds=gaf.Authenticate_Google(r"/home/jupyter/reusable_code/") #GAF is a package steve created with a list of useful functions
bq = bigquery.Client(project='itv-bde-analytics-dev',credentials=creds) #Apply credentials to BQ client "bq"

# Read in the FTS table created in BQ
query="""
select 
  timestamp_trunc(start,WEEK) as reportingDate
  , count(*) as vol 
from `itv-bde-analytics-dev.britbox_sandbox.ss_entitlements` 
where eventSubType.reportingEvent = 'A' and billingProvider in ('Stripe', 'iTunes') 
and timestamp_trunc(start,WEEK)<timestamp_trunc(current_timestamp(),WEEK)
group by 
  1
"""
FTS_df = bq.query(query).to_dataframe()

# Change the column names
FTS_df = FTS_df.rename(columns={ 'reportingDate':'reporting date','vol':'N'})

# Indexes the date column 
TS = FTS_df.set_index("reporting date")
TS=TS.tz_localize(None) # Remove timezone from FTS_df index

TS=pd.Series(TS['N'])
TS

### 2) Clean up the data & generate training/test sets

In [23]:
# Interpolate any missing days
TS=Interpolate_Missing(TS,freq='w')


In [24]:
# Cap outliers
TS_capped=Cap_data(TS,show_info=False)      


In [30]:
TS_test, TS_fullTrain, TS_recentTrain=generate_test_trains(TS,HoldoutPC=0.10,upperCutoff='2020-11-15')
TS_capped_test, TS_capped_fullTrain, TS_capped_recentTrain=generate_test_trains(TS_capped,HoldoutPC=0.10,upperCutoff='2020-11-15')

### 3) Visual inspection of the data

In [31]:
timeSeries_diagnostics(TS)

### 4) Grid Search across iterations of Holt Winters
Combinations include: 
* Additive & Multiplicative for the trend element
* Seasonal element Y/N, and if Yes, Additive and Multiplicative
* Different seasonal lags to try (if desired, change the  "seasonal" list. Default is 7 only

Additionally there are up to 4 datasets to train on
* The Full data
* The Recent data (i.e. the subset of full data to take more recent obs only)
* The Full data, with outliers removed
* The Recent data, with outliers removed

Obviously some of these datasets may be duplicates of each other, e.g. if the "recent" threshold covers all time, or if no outliers were detected. If so, these should be removed.


In [32]:
grid=HW_grid(seasonal=[2]) # Create a Holt-Winters hyperparameter search grid
training_sets={'Full training':TS_fullTrain, 'Recent training':TS_recentTrain,'Full training (capped)':TS_capped_fullTrain, 'Recent training (capped)':TS_capped_recentTrain}
training_sets=Remove_duplicate_training_sets(training_sets) # Run a function to exclude any training datasets that may be copies

### Steps to fit the models:
##### 0) Loop through training sets and model hyperparameter grid to enable next step
#####  1) Define the model's hyperparameters & training set
##### 2) Fit the model to return the optimised parameters
##### 3) Use the fitted model to forecast N steps ahead, where N is the size of the test/validation set
##### 4) Evaluate the N step model fit (MAPE, RMSE, etc.)

##### 5) Use the fitted model to then forecast N x 1-step forecasts (i.e. with up to date data, how well does it predict tomorrow).
 Importantly for this step, the paramaters are now fixed, but the trends etc. can be updated with real data. This is in contrast to step 3 & 4 where data cuts off at the end of the training 
 set, and thereafter the data is simply using past predictions to model future predictions, eventually resulting in a uniform pattern
##### 6) Evaluate the N x 1-step model fit (MAPE, RMSE, etc.)
##### 7) Store all the results etc. in a dictionary, and append to a list of all iterations





In [33]:
datafreq='w' # Define data frequency
hw_models=[] # Initialise 

#### 0) Loop through training sets and model hyperparameter grid to enable next step
for ser in training_sets: # For each training set
    for hyperparams in grid: # For each hyperparameter combo
        print("Fitting {} on dataset {}".format(hyperparams,ser))
        model =None
        model_fit=None
        #####  1) Define the model's hyperparameters & training set. NB this doesn't "run" the model.
        if hyperparams[1]==None:
            model=ExponentialSmoothing(training_sets[ser], trend=hyperparams[0],freq=datafreq)
        else:
            model=ExponentialSmoothing(training_sets[ser], trend=hyperparams[0],seasonal=hyperparams[1]\
                                       ,seasonal_periods=hyperparams[2],freq=datafreq)
            
        ##### 2) Fit the model to return the optimised parameters
        print('Optimising...')
        model_fit=model.fit() 
        #print(model_fit.mle_retvals)
        ##### 3) Use the fitted model to forecast N steps ahead, where N is the size of the test/validation set
        nStepPred=model_fit.model.predict(params=model_fit.params,start=TS_test.index.min(),end=TS_test.index.max())

        ##### 4) Evaluate the N step model fit (MAPE, RMSE, etc.)
        nStepfit=error_calcs(TS_test,nStepPred) 
        nStepfit['AIC']=model_fit.aic # Add AIC

       ##### 5) Use the fitted model to then forecast N x 1-step forecasts (i.e. with up to date data, how well does it predict tomorrow).
        newTrain=training_sets[ser].append(TS_test) # Append the test data onto the existing training set
        
        # In StatsModels prior to v 0.12 you set all the params in the model.fit, but couldn't pass seasonal starting parameters
        # Since v 0.12 you set the initialisation upfront in the model= statement, and the use_boxcox parameter, but the rest are set in the model.fit, just to confuse you
        
        paramdictModel= {key: value for key, value in model_fit.params.items() if (re.split('_',key)[0]=='initial' or key in ['use_boxcox'])}
        paramdictFit= {key: value for key, value in model_fit.params.items() if key not in [key for key in paramdictModel]+['lamda']}
        try:
            paramdictModel['initial_seasonal'] = paramdictModel.pop('initial_seasons')
        except:
            pass
        # Redefine the model to point at the new, longer training set ("newTrain")
        if hyperparams[1]==None:
            paramdictModel={key: value for key, value in paramdictModel.items() if len(re.findall('season',key))==0} # Remove seasonal parameters for non-seasonal models (they're empty anyway but still passed in the results Wrapper)
            paramdictFit={key: value for key, value in paramdictFit.items() if len(re.findall('season',key))==0} # Remove seasonal parameters for non-seasonal models
            
            retrained_model=ExponentialSmoothing(newTrain, trend=hyperparams[0],freq=datafreq,initialization_method='known', **paramdictModel)
        else:
            
            
            retrained_model=ExponentialSmoothing(newTrain, trend=hyperparams[0],seasonal=hyperparams[1]\
                                       ,seasonal_periods=hyperparams[2],freq=datafreq,initialization_method='known', **paramdictModel)
        print('Refitting without optimisation...')
        refitted_model=retrained_model.fit(**paramdictFit,optimized=False) # Apply the already-optimised parameters to the new train+test dataset
        nx1StepPred=refitted_model.model.predict(refitted_model.params,start=TS_test.index.min(),end=TS_test.index.max()) # Generate predictions
        
        ##### 6) Evaluate the N x 1-step model fit (MAPE, RMSE, etc.)
        nx1Stepfit=error_calcs(TS_test,nx1StepPred)




        ##### 7) Store all the results etc. in a dictionary, and append to a list of all iterations
        model_dict={'Trained_on':ser,\
                    'hyperparameters':hyperparams,\
                   'Fitted Model':model_fit,\
                   'Test size (N)':len(TS_test),\
                    'Model summary':model_fit.summary(),\
                   'Output parameters':model_fit.params,\
                    'Retraining parameters (Model)':paramdictModel, # Same as the output parameters, but parsed to form the inputs needed at the MODEL stage for refitting\
                    'Retraining parameters (Fit)':paramdictFit, # Same as the output parameters, but parsed to form the inputs needed at the FIT stage for refitting\
                    'AIC':model_fit.aic,\
                    'N-step fit':nStepfit,\
                    'N x 1-step fit':nx1Stepfit,\
                    'Refitted Params (QA)':refitted_model.params} # Verify they are indeed the same!
                   
        hw_models.append(model_dict)
        
        


### 5) Select best model and use for actual forecast (rather than test forecast)

In [48]:
best_AIC=min([x['AIC'] for x in hw_models])
best_Nstep=min([x['N-step fit']['MAPE'] for x in hw_models])
best_1step=min([x['N x 1-step fit']['MAPE'] for x in hw_models])
#best_model=[x for x in hw_models if x['N x 1-step fit']['MAPE']==best_1step][0]
#best_model=[x for x in hw_models if x['N-step fit']['MAPE']==best_Nstep][0]
best_model=[x for x in hw_models if x['AIC']==best_AIC][0]

# Check parameters were applied correctly
print(best_model['Refitted Params (QA)'])
print(best_model['Output parameters'])
best_model

In [39]:

FinalTraining=training_sets[best_model['Trained_on']].append(TS_test)
if best_model['hyperparameters'][1]==None:
    final_model=ExponentialSmoothing(FinalTraining, trend=best_model['hyperparameters'][0],freq=datafreq,initialization_method='known',**best_model['Retraining parameters (Model)'])
else:
    final_model=ExponentialSmoothing(FinalTraining, trend=best_model['hyperparameters'][0],seasonal=best_model['hyperparameters'][1]\
                               ,seasonal_periods=best_model['hyperparameters'][2],freq=datafreq,initialization_method='known',**best_model['Retraining parameters (Model)'])


fitted_final_model=final_model.fit(**best_model['Retraining parameters (Fit)'],optimized=False) # Apply the already-optimised parameters to the new train+test dataset

predictions=fitted_final_model.model.predict(fitted_final_model.params,start=FinalTraining.index.min(),end=TS.index.max()) # Generate predictions


In [40]:

comparedf=pd.DataFrame(TS[FinalTraining.index.min():])
comparedf['predicted']=predictions
comparedf.plot(ylim=(0))

In [43]:
forecasts=fitted_final_model.model.predict(fitted_final_model.params,start=FinalTraining.index.max(),end='2021-01-01') # Generate predictions
forecasts

In [47]:
fc=pd.Series(forecasts)
fc.plot(ylim=(0))

In [50]:
fc[1:-1].sum()